In [16]:
import pandas as pd
pd.set_option('display.max_columns', 300)
from datetime import datetime
import numpy as np

## Step 1: Read in hold out data, scalers, and best model

In [23]:
### making new notebooks
import pickle
pickle_in = open("model.pickle","rb")
model = pickle.load(pickle_in)
scaler = pickle.load(open("scaler.pickle", 'rb'))

In [24]:
holdout = pd.read_csv('kc_house_data_test_features.csv', index_col=0)

In [25]:
def clean_data(test_df):
    test_df.date = test_df.date.apply(lambda x: x[0:8])
    test_df['yr_sold'] = test_df.date.apply(lambda x: datetime.strptime(x, "%Y%m%d").year)
    test_df = test_df.drop(axis=1, columns=['id', 'date', 'floors'])
    test_df['log_sqft_living'] = np.where(test_df['sqft_living'], np.log(test_df['sqft_living']), test_df['sqft_living'])
    test_df.drop(columns=['sqft_lot'], axis=1, inplace=True)
    test_df.drop(columns=['sqft_living'], inplace=True, axis=1)

    # Create is_renovated column: if house is equal to or older than 2000, then it is renovated.

    test_df['is_renovated']=np.where(test_df['yr_renovated'] >= 2000, 1, 0)
    test_df['yr_old'] = np.where(test_df['yr_built'], datetime.now().year - test_df['yr_built'], test_df['yr_built'])
    test_df['is_viewed'] = np.where(test_df['view'] > 0, 1, 0)




    # np.log on skewed_cols
    test_df['log_sqft_above'] = np.where(test_df['sqft_above'], np.log(test_df['sqft_above']), test_df['sqft_above'])
    test_df['log_sqft_basement'] = np.where(test_df['sqft_basement'], np.log(test_df['sqft_basement']), test_df['sqft_basement'])
    test_df['log_sqft_living15'] = np.where(test_df['sqft_living15'], np.log(test_df['sqft_living15']), test_df['sqft_living15'])

    skewed_cols = ['sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15']
    test_df.drop(columns=skewed_cols, axis=1, inplace=True)
    test_df.drop(columns=['yr_built'], inplace=True, axis=1)

    test_df = pd.concat([test_df, pd.get_dummies(test_df['zipcode'], drop_first=True)], axis=1)
    test_df.drop('zipcode', axis=1, inplace=True)

    test_df.columns = test_df.columns.map(lambda x: 'zip_' + str(x) if type(x) != str else x)
    return test_df

In [26]:
df = clean_data(holdout)

In [27]:
df = scaler.fit_transform(df)

In [ ]:
# final_scaler = read_pickle(filename)
# final_model = read_pickle(filename)

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values and scaling) that we performed on the original data.  

In [ ]:
# transformed_holdout = final_scaler(holdout)

## Step 3: Predict the holdout set

In [30]:
final_answers = model.predict(df)
final_answers

array([586112.75687086, 614418.08624037, 423299.90115152, ...,
       186232.22111591, 441609.29005293, 187937.18551852])

In [31]:
final_answers = pd.DataFrame(final_answers)

In [32]:
final_answers

,0
0,586112.756871
1,614418.086240
2,423299.901152
3,381387.778570
4,532191.621249
...,...
4318,507389.649801
4319,466480.411032
4320,186232.221116
4321,441609.290053


## Step 4: Export your predictions

In [34]:
final_answers.to_csv('WH-AN_prediction.csv')